# Create an AI pipeline for quiz generation, using LangGraph

One way to boost customer engagement in the news industry is with news quizzes. The quiz would ask questions about the articles the user recently read.

In this hands-on exercise, we will implement a quiz generator. To achieve this, we will use generative AI, retrieval augmented generation, and the LangGraph library. 

Here is an overview for the AI pipeline:
- The user specifies a **topic** of interest
- Search for corresponding news articles in the **Reuters dataset**
- The user selects 1 from the top-3 new articles
- Generate quiz which will have well-defined format

![AI pipeline](assets/ai_pipeline.png "AI pipeline for quiz generation")

In [ ]:
# Install necessary dependencies
# - this takes ~3 minutes, give it some patience
# - the imports can show error messages, you can ignore them

!pip install unsloth
!pip install langchain
!pip install langgraph
!pip install langchain_huggingface

In [ ]:
# Do all necessary imports

from typing import List, Dict
from typing_extensions import TypedDict
from langchain_core.documents import Document
from langchain_core.vectorstores import InMemoryVectorStore
from langgraph.types import Command, interrupt
from langgraph.graph import START, StateGraph, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_huggingface import HuggingFaceEmbeddings
from IPython.display import Image, display
from datasets import load_dataset
import pandas as pd
from unsloth import FastLanguageModel
from transformers import PreTrainedTokenizer, PreTrainedModel

## Prepare LLM for answer generation

We use [Unsloth](https://docs.unsloth.ai) for LLM inference. If you prefer to use an LLM API instead, feel free to adjust the Notebook accordingly. Note that other parts of this workshop will also use Unsloth. 

In [ ]:
# Instantiate Unsloth model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
)
FastLanguageModel.for_inference(model)

In [ ]:
# We use this helper function for LLM inference. You can leave it as-is.

def llm_inference(
        messages: List[Dict],
        model: PreTrainedModel,
        tokenizer: PreTrainedTokenizer
) -> str:
    """
    :param messages: Messages for the LLM 
    :param model: The initialized unsloth LLM
    :param tokenizer: The pretrained tokenizer
    :return: The LLM answer as raw string
    """
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    input_tokens = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")
    input_len = len(input_tokens.tokens())
    output_tokens = model.generate(**input_tokens)
    output_clipped = output_tokens[:, input_len:-1]
    result = tokenizer.batch_decode(output_clipped)
    return result[0]

## Generate your first quiz

To start off, we'll create our first quiz. For demo purposes, we'll use one hard-coded news article. The more important part is the LLM prompt. In the prompt, we give instructions for quiz generation. Moreover, we define the output format, which the LLM hopefully adheres to during quiz generation. 

In [ ]:
# Define a helper function for creating the LLM messages.
def create_messages(news_article: str) -> List[Dict]:
    """
    :param news_article: The news article 
    :return: A quiz about the news article
    """
    messages = [
        {
            "role": "system",
            "content": """Please generate one multiple choice quiz for the provided news article.
            
            The quiz should have the following format:
            
            [Question]
            
            [Choice 1]
            [Choice 2]
            [Choice 3]
            
            [Solution]
            """,
        },
        {
            "role": "user",
            "content": f"Here is the news article: {news_article}"
        }
    ]
    return messages

In [ ]:
# Create quiz
news_article = "U.S. Agriculture Secretary Richard Lyng said he would not agree to an extension of the 18-month whole dairy herd buyout program set to expire later this year. Speaking at the Agriculture Department to representatives of the U.S. National Cattlemen\'s Association, Lyng said some dairymen asked the program be extended. But he said the Reagan administration, which opposed the whole herd buyout program in the 1985 farm bill, would not agree to an extension. The program begun in early 1986, is to be completed this summer. U.S. cattlemen bitterly opposed the scheme, complaining that increased dairy cow slaughter drove cattle prices down last year. Reuter"
# TODO: generate messages for LLM
# Use the above defined function "create_messages()".
# >>>>>>>>>>>>>>>>>>>>>>>>>
messages: str = ...
# <<<<<<<<<<<<<<<<<<<<<<<<<
# TODO: query the LLM for quiz generation
# Use the above defined function "llm_inference()"
# >>>>>>>>>>>>>>>>>>>>>>>>>
llm_response: str = ...
# <<<<<<<<<<<<<<<<<<<<<<<<<
# TODO: Investigate the generated quiz. Does it have the desired scope and format?

## Prompt adjustments

Next, we'll try to tweek the messages for the LLM.

In [ ]:
# TODO: adjust prompt
def create_messages_more_answers(news_article: str) -> List[Dict]:
    # Adjust the messages for the LLM, such that the generated quiz has 5 instead of 3 answers.
    # >>>>>>>>>>>>>>>>>>>>>>>>>
    pass
    # <<<<<<<<<<<<<<<<<<<<<<<<<
messages: str = create_messages_more_answers(news_article)
# <<<<<<<<<<<<<<<<<<<<<<<<<
# TODO: re-query the LLM for quiz generation
# Use the above defined function "llm_inference()"
# >>>>>>>>>>>>>>>>>>>>>>>>>
llm_response: str = ...
# <<<<<<<<<<<<<<<<<<<<<<<<<

In [ ]:
# TODO: Can you think of any other way to adjust quiz generation?
# - Increase difficulty
# - ... any other ideas?

## Load Reuters dataset

Throughout this notebook, we'll be using the [Reuters news dataset](https://huggingface.co/datasets/ucirvine/reuters21578) from Hugging Face.
We download it below. This dataset contains short articles from Reuters' financial newswire service from 1987. 

In [ ]:
# Load dataset
# - if asked to run custom code, type "y" for YES.
reuters_ds = load_dataset('ucirvine/reuters21578','ModHayes')
news_raw = reuters_ds["train"].to_pandas()
print(f"Loaded {len(news_raw)} news articles.")

## Preprocess news articles

First we perform some preprocessing on the news data. We'll store all articles in a [pandas DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). For each article, we keep the actual news text, plus the news title. On the resulting strings, we remove unwanted characters.

In [ ]:
# Merge title and text, drop unnecessary columns
news_raw["title_and_text"] = news_raw['title'] + ' | ' + news_raw['text']
news = news_raw[["title_and_text", "date", "places"]]

In [ ]:
# Clean up text, remove unnecessary characters
pd.options.mode.chained_assignment = None
news["title_and_text"] = news.apply(lambda x: x["title_and_text"].replace("\\n", " "), axis=1)
news["title_and_text"] = news.apply(lambda x: x["title_and_text"].replace("\\\"", "\""), axis=1)
news["title_and_text"] = news.apply(lambda x: " ".join(x["title_and_text"].split()), axis=1)
news.head()

## Create RAG database

We want to support quiz generations for a user specified topic. For this reason, we create a vector store, where one can query news articles by topic.

In [ ]:
# Setup RAG vector store
# - This can take ~ 1 minute, give it some patience
texts_to_encode = news['title_and_text'].to_list()
embedder = HuggingFaceEmbeddings(
    model_name = "sentence-transformers/all-MiniLM-L6-v2"
)
vectorstore = InMemoryVectorStore.from_texts(texts=texts_to_encode, embedding=embedder)

In [ ]:
# Search articles about a given topic
query = "An article on agriculture"
k=3
# TODO: use the vector store above to search for 3 news articles corresponding to "agriculture"
# Hint: use InMemoryVectorStore's "similarity_search()" function
# >>>>>>>>>>>>>>>>>>>>>>>>>
results: List[Document] = ...
# <<<<<<<<<<<<<<<<<<<<<<<<<
# TODO: verify the articles are really about "agriculture"

In [ ]:
# TODO: Check whether you can search for articles about "coffee"
# >>>>>>>>>>>>>>>>>>>>>>>>>
...
# <<<<<<<<<<<<<<<<<<<<<<<<<

## Create AI pipeline with LangGraph

We now create a graph for quiz generation. Note that the graph is deterministic and doesn't use tools.
Since time is short, we provide all code for creating the graph. Checkout more about LangGraph in their [official documentation](https://langchain-ai.github.io/langgraph/tutorials/introduction/).
Our graph leverages the following features:
- Maintain a conversation state
- Human in the loop

In [ ]:
class State(TypedDict):
    """The state during graph traversal."""
    topic: str
    documents_from_lookup: List[Document]
    selected_document: Document
    quiz: str

def retrieve(state: State):
    """Search for news articles according to the topic selected by the user."""
    documents_from_lookup = vectorstore.similarity_search(state["topic"], k=3)
    return {"documents_from_lookup": documents_from_lookup}

def human_feedback(state: State):
    """Let the user choose one article, on which the quiz will be based."""
    article_selection = interrupt("Let user choose article")
    selected_document=state["documents_from_lookup"][int(article_selection)]
    return {"selected_document": selected_document}

def generate(state: State):
    """Generate a quiz."""
    news_article = state["selected_document"].page_content
    messages = create_messages(news_article)
    llm_response = llm_inference(messages, model, tokenizer)
    return {"quiz": llm_response}

# Build the graph with all nodes and edges
builder = StateGraph(State)
builder.add_node("retrieve", retrieve)
builder.add_node("human_feedback", human_feedback)
builder.add_node("generate", generate)
builder.add_edge(START, "retrieve")
builder.add_edge("retrieve", "human_feedback")
builder.add_edge("human_feedback", "generate")
builder.add_edge("generate", END)

memory = MemorySaver()
graph = builder.compile(checkpointer=memory)
thread = {"configurable": {"thread_id": "1"}}

display(Image(graph.get_graph().draw_mermaid_png()))

## Run the graph

We now run the AI pipeline to generate the quiz. Note this is interactive. The user first needs to select a topic, and later needs to choose an article from a list of proposals.

In [ ]:
# Run the graph from the start, until user selection step
topic = input("Please select your topic: ")
initial_input = {"topic": topic}
for event in graph.stream(initial_input, thread, stream_mode="updates"):
    pass
# Display article options
article_options=graph.get_state(config=thread).values['documents_from_lookup']
print("Article candidates:")
for id, doc in enumerate(article_options):
    content = doc.page_content
    print(f"[{id}] {content}")

In [ ]:
# TODO: have a look at the retrieved news articles, which one is most suitable for quiz generation?

In [ ]:
# Get human feedback
human_feedback = input("Please select which article you'd like to use [0,1,2]: ")

# Continue the graph execution
for event in graph.stream(
        Command(resume=human_feedback), thread, stream_mode="updates"
):
    pass

# Show final quiz
quiz=graph.get_state(config=thread).values['quiz']
print(quiz)

In [ ]:
# TODO: Review the quiz

# Adjust AI pipeline

In [ ]:
# TODO: Make the quiz more entertaining. Here are some improvement ideas:
# - Show the original news article above the quiz, when displaying the quiz to the user.
# - The final output should include the year of the news article.